In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch 
from torch import nn
from torch.utils.data import Dataset, DataLoader

from scipy import ndimage as ndi
from scipy import fft, ifft
import scipy

from skimage import data
from skimage import io, transform, color
from skimage.util import img_as_float, pad
from skimage.filters import gabor_kernel

import pickle 
import os
import sys
import h5py
import warnings
import time
import json
import nibabel as nib

import gfb_utils as g
import nsd_utils as n
import model_fitting as m

sys.path.append('/user_data/mmhender/coco_annot/cocoapi/PythonAPI/')
from pycocotools.coco import COCO


In [3]:
class gfb_encoding_model(nn.Module):
    """
    Definition of a simple linear encoding model that can be fit with grad descent.
    nFeats would be e.g. number of feature maps. 
    """
    def __init__(self, nFeats, nVoxels, nRFs):
    
        super(gfb_encoding_model, self).__init__()
#         self.bank = filter_bank
        self.nFeats = nFeats
        self.nVoxels = nVoxels
        self.nRFs = nRFs
        self.weights = nn.Parameter(torch.randn(nFeats, nVoxels, nRFs) / np.sqrt(nFeats*nVoxels*nRFs))
        self.bias = nn.Parameter(torch.zeros(nVoxels, nRFs))
        
    def forward(self, x):
        
        nTrials = x.shape[0]
        out = torch.zeros(nTrials, self.nVoxels, self.nRFs)
        for rf in range(nRFs):   
            out[:,:,rf] = torch.matmul(x[:,:,rf], self.weights[:,:,rf]) + self.bias[:,rf]
        pred_resp = out
#         pred_resp = torch.matmul(x, self.weights) + torch.tile(self.bias,[nTrials, 1]) 
        return pred_resp

In [7]:
## Making the filters here:

# First define the feature bank itself
freqs_cpp = np.round(np.logspace(np.log10(0.02), np.log10(0.25), 6),2)
orient_step = 5
orients_deg = np.arange(0,181-orient_step, orient_step)
spat_freq_bw = 1
spat_aspect_ratio = 1
n_sd_out = 7

# Specify stuff about images
process_at_size = [200,200]

bank = g.filter_bank(orients_deg, freqs_cpp, spat_freq_bw, spat_aspect_ratio, n_sd_out, image_size = process_at_size)
nsd_inds = [0,1]
all_feat = g.get_nsd_gabor_feat(nsd_inds, bank)
                            
print(np.shape(all_feat))    

# Create set of candidate RFs
sizes = np.arange(5, 21, 5)
x_centers = np.arange(20, 181, 20)
y_centers = np.arange(20, 181, 20)

rf_stack, x_list, y_list, size_list  = g.get_rf_stack(x_centers, y_centers, sizes, process_at_size)

nRFs = np.shape(rf_stack)[2]
nFeats = np.shape(all_feat)[3]

# Making a full matrix, [nIms x nFeatures x nRFs]
# feats_by_rfs = g.get_feats_by_rfs(all_feat, rf_stack)

# print(np.shape(feats_by_rfs))
# nFeats = np.shape(feats_by_rfs)[1]
# nRFs = np.shape(feats_by_rfs)[2]

  0%|          | 0/2 [00:00<?, ?it/s]

size of filter stack will be:
(600, 600, 216)


100%|██████████| 2/2 [00:24<00:00, 12.37s/it]


(2, 200, 200, 216)


In [5]:
curr_batch_trn['subject_df']

NameError: name 'curr_batch_trn' is not defined

In [8]:
## Set up training skeleton


# First info abt subject, ROI to look at
subj = 1
roi_label = 1
roi_filename='lh.prf-visualrois.nii.gz'
roi_voxels = n.get_roi_voxels(subj, roi_label, roi_filename)
nVox = np.shape(roi_voxels)[1]

dset = n.nsd_dataset(subj, voxel_inds = roi_voxels)
trn, val, _ = n.get_dataset_splits(dset)
val_batch_size = 2
trn_batch_size = 50
valgenerator = n.nsd_dataloader(val, batch_size=val_batch_size, shuffle=True, rndseed = 858875)
trngenerator = n.nsd_dataloader(trn, batch_size=trn_batch_size, shuffle=True, rndseed = 543545)

curr_batch_trn = next(trngenerator)
# Data for this batch, trials x voxels
dat2fit = curr_batch_trn['data']
# Which images were shown on these trials? get indices into NSD 0-73000
nsd_inds = curr_batch_trn['subject_df']['nsdId']

h5py loading time is 6.10
h5py loading time is 5.85
h5py loading time is 6.30
h5py loading time is 5.75
h5py loading time is 7.89
h5py loading time is 9.64
h5py loading time is 9.53
h5py loading time is 11.62
h5py loading time is 9.44
h5py loading time is 7.37
h5py loading time is 6.48
h5py loading time is 6.62
h5py loading time is 6.69
h5py loading time is 7.29


In [9]:
# Get features for these images based on gabor filter bank (already defined)
print('getting features for this batch')
all_feat = g.get_nsd_gabor_feat(nsd_inds, bank)    

# Combine these features with all the possible RFs
feats_by_rfs = g.get_feats_by_rfs(all_feat, rf_stack)
x = feats_by_rfs

  0%|          | 0/50 [00:00<?, ?it/s]

getting features for this batch


100%|██████████| 50/50 [06:51<00:00,  8.23s/it]


In [10]:
lr = 10e-12

model = gfb_encoding_model(nFeats, nVox, nRFs).double()

loss_all = np.zeros([nRFs, nVox])

max_steps = 50

# counting total passes over the entire training set (nepochs = nsteps/nbatches)
nepochs_done = 0 
   
# loop over training steps 
# one step=one weight update, based on one batch of data
for step in range(max_steps):

    print('step %d - getting batch ready'%step)
#     # first get the next batch of data from trn set
#     try:
#       # try taking next sample from current iterator object
#       curr_batch_trn = next(trngenerator)
#     except StopIteration:
#       # if the previous iterator object is exhausted, then the epoch is finished, 
#       # need to make a new iterator for next epoch.
#       nepochs_done = nepochs_done+1;
#       trngenerator = n.nsd_dataloader(trn, batch_size=trn_batch_size, shuffle=True, rndseed = (543545+n_epochs_done))
#       curr_batch_trn = next(trngenerator)

#       print('STARTING EPOCH %d'%nepochs_done)
    
#     # Data for this batch, trials x voxels
#     dat2fit = curr_batch_trn['data']
    
#     # Which images were shown on these trials? get indices into NSD 0-73000
#     nsd_inds = curr_batch_trn['subject_df']['nsdId']
    
#     # Get features for these images based on gabor filter bank (already defined)
#     print('getting features for this batch')
#     all_feat = g.get_nsd_gabor_feat(nsd_inds, bank)    
    
#     # Combine these features with all the possible RFs
#     feats_by_rfs = g.get_feats_by_rfs(all_feat, rf_stack)
#     rf=0
#     x = feats_by_rfs
    
    # first get predictions and loss for the current weights.
    pred_resp = model(x) 
    loss = loss_sse(pred_resp, np.tile(dat2fit[:,:,np.newaxis], [1,1,nRFs]))
    loss_all = loss.detach().numpy()
        
    # Summing over all voxels and RFs. Because need a scalar value for loss.
    # If we minimize this value, it is same as minimizing for one voxel/RF at a time.
    # This works because the weights that contribute to a given voxel and a given RF are 
    # restricted to one column of weight matrix.
    summed_loss = torch.sum(loss)
    
    print('step %d, summed loss is %.2e'%(step, summed_loss.item()))
    
    # run the graph backward, to get gradient values. 
    summed_loss.backward()
#     print('one selected units gradient is %.6f'%model.weights.grad[0,0,0])
#     print('weight of that unit before update is %.6f'%model.weights[0,0,0])
    
    with torch.no_grad():
        # update weights based on this learning step
        model.weights -= model.weights.grad * lr
        model.bias -= model.bias.grad * lr
        # zero the gradients out before next step
        model.weights.grad.zero_()
        model.bias.grad.zero_()

#     print('weight of that unit after update is %.6f'%model.weights[0,0,0])
    



step 0 - getting batch ready


NameError: name 'loss_sse' is not defined

In [ ]:
print(np.shape(loss_all))
best_spat_rf = np.argmin(loss_all,axis=1)
best_spat_rf.shape

dat_pred = pred_resp.detach().numpy()

nTrials = np.shape(dat_pred)[0]

best_pred_resp = np.zeros([nTrials, nVoxels])
for vv in range(nVoxels):

    best_pred_resp[:,vv] = dat_pred[:,vv,best_spat_rf[vv]]
    


In [ ]:
np.shape(dat_pred)

In [122]:
print('%.2e'%0.0001)

1.00e-04


In [47]:
loss_all[rf,:] = loss.detach().numpy()
loss_all

array([[41780.29431899, 33562.17129152, 12991.80968866, ...,
         8182.211206  , 13622.49080108,  5935.08898458],
       [    0.        ,     0.        ,     0.        , ...,
            0.        ,     0.        ,     0.        ],
       [    0.        ,     0.        ,     0.        , ...,
            0.        ,     0.        ,     0.        ],
       ...,
       [    0.        ,     0.        ,     0.        , ...,
            0.        ,     0.        ,     0.        ],
       [    0.        ,     0.        ,     0.        , ...,
            0.        ,     0.        ,     0.        ],
       [    0.        ,     0.        ,     0.        , ...,
            0.        ,     0.        ,     0.        ]])

In [ ]:
    
def loss_sse(pred_resp, real_resp):
    """ 
    Calculate loss based on sum of squared error.
    If input is a matrix, assume [nTrials x nVoxels] and returns [nVoxels]
    """
    
    if not isinstance(pred_resp, torch.Tensor):
        pred_resp = torch.tensor(pred_resp)
    if not isinstance(real_resp, torch.Tensor):
        real_resp = torch.tensor(real_resp)

    if len(torch.squeeze(pred_resp).shape)==1 & len(torch.squeeze(real_resp).shape)==1:
        pred_resp = torch.squeeze(pred_resp)
        real_resp = torch.squeeze(real_resp)       
        error = torch.sum(torch.square(pred_resp - real_resp))
    else:             
        assert(np.all(pred_resp.shape==real_resp.shape))
        error = torch.sum(torch.square(pred_resp - real_resp), axis=0)
    
    return error

In [3]:
## Making the filters here:

# First define the feature bank itself
freqs_cpp = np.round(np.logspace(np.log10(0.02), np.log10(0.25), 3),2)
orient_step = 45
orients_deg = np.arange(0,181-orient_step, orient_step)
spat_freq_bw = 1
spat_aspect_ratio = 1
n_sd_out = 7

# Specify stuff about images
process_at_size = [200,200]

bank = g.filter_bank(orients_deg, freqs_cpp, spat_freq_bw, spat_aspect_ratio, n_sd_out, image_size = process_at_size)
all_feat = g.get_nsd_gabor_feat(nsd_inds, bank)
                            
print(np.shape(all_feat))    

# Create set of candidate RFs
sizes = np.arange(5, 21, 5)
x_centers = np.arange(20, 181, 20)
y_centers = np.arange(20, 181, 20)

rf_stack, x_list, y_list, size_list  = g.get_rf_stack(x_centers, y_centers, sizes, process_at_size)

# Making a full matrix, [nIms x nFeatures x nRFs]
# feats_by_rfs = g.get_feats_by_rfs(all_feat, rf_stack)

# print(np.shape(feats_by_rfs))
# nFeats = np.shape(feats_by_rfs)[1]
# nRFs = np.shape(feats_by_rfs)[2]

  0%|          | 0/750 [00:00<?, ?it/s]

size of filter stack will be:
(600, 600, 12)


100%|██████████| 750/750 [05:29<00:00,  2.28it/s]


(750, 200, 200, 12)
torch.Size([750, 12, 324])


In [ ]:
nSamples = 10
nVox = 6
nSteps=50
lr = 0.000001

model = m.encoding_model(nFeats).double()

# for vv in range(nVox):
for vv in range(1):
    
    print('fitting model for voxel %d of %d'%(vv, nVox))
    
    # gather the real data we would like to fit, for a single voxel over many images
    real_resp = torch.tensor(dat2fit[:,vv])
    
    for rf in range(1):
#     for rf in range(nRFs):
        
        print('fitting model for rf position %d of %d'%(rf, nRFs))
        
        # grabbing features for each image, within this candidate RF
        x = feats_by_rfs[:,:,rf]

        for ss in range(nSteps):
            
            # first get predictions and loss for the current weights.
            pred_resp = model(x) 
            loss = m.loss_sse(pred_resp, real_resp)

            print('step %d, loss is %.2f'%(ss, loss))
        
            # run the graph backward, to get gradient values. 
            loss.backward()

            with torch.no_grad():
                # update weights based on this learning step
                model.weights -= model.weights.grad * lr
                model.bias -= model.bias.grad * lr
                # zero the gradients out before next step
                model.weights.grad.zero_()
                model.bias.grad.zero_()

In [23]:
# Some code to test ridge regression function w fake data

vv=0
rf=0
model = ridge_regression_model(lamb=0.001)
# make random trn data (these are features)
x_trn = torch.tensor(np.random.normal(0,1,[15,4]))
# random mapping from trn features to responses
rand_wts = torch.tensor(np.random.normal(0,1,[4,6]))
y_trn = x_trn @ rand_wts

# tst data is just trn data plus some noise
x_tst = x_trn + np.random.normal(0,0.5, np.shape(x_trn))
y_tst = y_trn + np.random.normal(0,0.5, np.shape(y_trn))

# fit and predict
model.fit(x_trn, y_trn)
y_hat = model.predict(x_tst)

print(np.shape(y_hat))

plt.figure();
plt.subplot(2,2,1)
plt.pcolormesh(y_tst)
plt.subplot(2,2,2)
plt.pcolormesh(y_hat) # plot predictions of voxel responses
plt.subplot(2,2,3)
plt.pcolormesh(rand_wts)
plt.subplot(2,2,4)
nFeats = np.shape(rand_wts)[0]
plt.pcolormesh(model.w[0:nFeats,:]) # plot computed weight matrix

tensor(467.5542, dtype=torch.float64, requires_grad=True)

In [ ]:
nSamples = 10
batch_size = 200
nVox = 6
nEpochs = 20
nSteps = 10
# nSteps = int(np.ceil(nSamples / batch_size * nEpochs))
lr = 1e-8

model = encoding_model(nFeats).double()

dat2fit = dat2use[0:nSamples,0:nVox]
# dat2fit = np.random.normal(0,1,[nSamples, nVox])

# first get features for all samples
nsd_inds = np.arange(0,nSamples)

# for vv in range(nVox):
for vv in range(1):
    
    print('fitting model for voxel %d of %d'%(vv, nVox))
    
    # gather the real data we would like to fit, for a single voxel over many images
    real_resp = torch.tensor(dat2fit[:,vv])
    
    for rf in range(1):
#     for rf in range(nRFs):
        
        print('fitting model for rf position %d of %d'%(rf, nRFs))
        
        # grabbing features for each image, within this candidate RF
        x = feats_by_rfs[:,:,rf]

        for ss in range(nSteps):
            
            # first get predictions and loss for the current weights.
            pred_resp = model(x) 
#             loss = torch.sqrt(loss_sse(pred_resp, real_resp))
            loss = loss_sse(pred_resp, real_resp)
    
            print('step %d, loss is %.2f'%(ss, loss))
            
            
            # run the graph backward, to get gradient values. 
            loss.backward()

            print(model.weights.grad[0])
            print(model.weights[0])
            
            with torch.no_grad():
                # update weights based on this learning step
                model.weights -= model.weights.grad * lr
                model.bias -= model.bias.grad * lr
                # zero the gradients out before next step
                model.weights.grad.zero_()
                model.bias.grad.zero_()
                
            print(model.weights[0])
            